In [ ]:
!pip install datasets
!pip install transformers

from datasets import load_dataset
from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification
import tensorflow as tf

# NSMC 데이터셋을 가져오기
dataset = load_dataset("nsmc")

# train_data와 test_data를 추출하기
train_data = dataset["train"]
test_data = dataset["test"]

# 텍스트와 레이블 추출
train_texts = train_data["document"]
train_labels = train_data["label"]
test_texts = test_data["document"]
test_labels = test_data["label"]

In [ ]:
# Bert 토크나이저를 가져오기
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# train 과 test 데이터셋을 토큰화
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
# 데이터를 텐서플로우 데이터셋으로 변환
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [ ]:
# Bert 모델 가져오기
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased")

# 모델 컴파일
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# 모델 학습
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=64, validation_data=test_dataset.batch(16))